In [1]:
import os
import sys
import time
import random
import numpy as np
import pandas as pd
from pathlib import Path
import cv2
import glob
from typing import List
from tqdm.auto import tqdm

# from solution import predict

## solution.py текущий
это то что отправляется в архиве с моделью - solution.py 

Тебе надо поменять только эту ячейку сохранив функцию predict

In [3]:
import importlib.util
from pathlib import Path

sol_path = Path('solution/yolo11m-v4-1536-25e-new-sol/solution.py').resolve()
spec = importlib.util.spec_from_file_location('uav_solution', str(sol_path))
sol = importlib.util.module_from_spec(spec)
spec.loader.exec_module(sol)
# Override predict used below to call our ORT solution
predict = sol.predict
print('predict wired from:', sol_path)



2025-08-09 12:50:18,124 | INFO | ONNX Runtime providers selected: ['CUDAExecutionProvider', 'CPUExecutionProvider']
2025-08-09 12:50:18,369 | INFO | ONNX Runtime active providers: ['CUDAExecutionProvider', 'CPUExecutionProvider']
2025-08-09 12:50:18,726 | INFO | Warm-up done.


predict wired from: /home/qwerty/github/archipelago2035/solution/yolo11m-v4-1536-25e-new-sol/solution.py


## Далее фаилы от организаторов (функции они выложили на портале)

In [5]:
# [ Параметры ]
IMAGE_EXTS = ('.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG')
COLUMNS = ['image_id', 'label', 'xc', 'yc', 'w', 'h', 'w_img', 'h_img', 'score', 'time_spent']

# Фиксируем сиды
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [6]:
#это фаил от огров
def process_images(images_path: str,
        result_csv_path: str = None) -> pd.DataFrame:
    """Функция обработки папки с изображениями, обрабатывает изображения по одному,
    отправляет их на функцию predict из файла solution.py (нужно переписать на свою),
    рассчитывает время инференса на изображение и записывает результат в pd.DataFrame
    
    Args:
        images_path (str): путь до директории с изображениями
        result_csv_path (str, optional): Путь, куда сохранять результат предсказаний. По-умолчанию None.

    Raises:
        Exception: Директория {images_dir} не найдена!
        Exception: Отсутствуют изображения в папке {images_dir}
        Exception: Время выполнения на тестовой выборке {time_spent} превышает {TIME_LIMIT_TEST // 60} минут!

    Returns:
        pd.DataFrame: датафрейм с результатом предсказаний
    """
    # Тестовая папка должна содержать подпапку images
    images_dir = os.path.join(images_path, 'images')
    # собираем все изображения по списку расширений
    image_paths = []
    for ext in IMAGE_EXTS:
        image_paths.extend(Path(images_dir).glob(f'*{ext}'))
    image_paths = sorted(set(image_paths))
    # Перемешиваем пути c изображениями
    random.shuffle(image_paths)
    images_count = len([p for p in Path(images_dir).iterdir() if p.suffix.lower() in [e.lower() for e in IMAGE_EXTS]])
    
    # Проверяем, существует ли директория, изображения
    assert os.path.exists(images_dir), Exception(f"Директория {images_dir} не найдена!")
    assert images_count > 0, Exception(f'Отсутствуют изображения в папке {images_dir}')

    results = []
    # Обрабатываем изображения по одному
    for image_path in image_paths:
        image_id = Path(image_path).stem
        # Открываем изображение в RGB формате
        image = cv2.imread(str(image_path), -1)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h_img, w_img, _ = image.shape

        # Засекаем время выполнения функции predict
        start_time = time.time()
        # Вызываем функцию predict для одного изображения
        image_results = predict([image])
        # print(image_results)
        # Останавливаем таймер
        elapsed_time = time.time() - start_time
        time_per_image = round(elapsed_time, 4)
        
        # Дополняем результаты ID изображения и затраченным временем
        if image_results and image_results[0]:
            for res in image_results[0]:
                res['image_id'] = image_id
                res['time_spent'] = time_per_image
                res['w_img'] = w_img
                res['h_img'] = h_img
                results.append(res)
        else:
            res = {'xc': None,
                   'yc': None,
                   'w': None,
                   'h': None,
                   'label': 0,
                   'score': None,
                   'image_id': image_id,
                   'time_spent': time_per_image,
                   'w_img': None,
                   'h_img': None
            }
            results.append(res)

    result_df = pd.DataFrame()
    if results and result_csv_path:
        result_df = pd.DataFrame(results, columns=COLUMNS)
        result_df = result_df.fillna(value=np.nan)
        result_df.to_csv(result_csv_path, index=False, na_rep=np.nan)
    print('Обработка выборки выполнена успешно!')
    
    return result_df

## Тут считаем ground true для расчета метрики

In [7]:
# Те же названия столбцов, что и в коде оценки
COLUMNS = ['image_id', 'label', 'xc', 'yc', 'w', 'h', 'w_img', 'h_img', 'score', 'time_spent']

def build_gt_csv_from_yolo(
        dataset_root: str,
        out_csv: str,
        images_subdir: str = 'images',
        labels_subdir: str = 'labels',
        image_exts: List[str] = ('.jpg', '.jpeg', '.png', ".JPG")
    ) -> pd.DataFrame:
    """
    Преобразует YOLO-разметку в CSV с колонками, совместимыми с вашим evaluate().
    
    dataset_root
        ├── images/
        └── labels/
    
    YOLO-формат строк в labels/*.txt:
        <class_id> <xc> <yc> <w> <h>  (все нормализовано в [0,1])
    """
    images_dir = Path(dataset_root) / images_subdir
    labels_dir = Path(dataset_root) / labels_subdir
    assert images_dir.exists(), f'images dir not found: {images_dir}'
    assert labels_dir.exists(), f'labels dir not found: {labels_dir}'

    rows = []
    # Соберем все изображения с нужными расширениями
    image_paths = []
    for ext in image_exts:
        image_paths.extend(list(images_dir.glob(f'*{ext}')))

    image_paths = sorted(image_paths)

    for img_path in image_paths:
        image_id = img_path.stem
        label_path = labels_dir / f'{image_id}.txt'

        # Получаем размеры изображения
        img = cv2.imread(str(img_path), -1)
        if img is None:
            print(f'[WARN] Could not read image: {img_path}')
            continue
        h_img, w_img = img.shape[:2]

        # Если файла нет или он пуст — это значит, что GT боксов для этого кадра нет
        if not label_path.exists() or os.path.getsize(label_path) == 0:
            # В GT можно просто ничего не добавлять для этого image_id — 
            # скрипт метрики корректно обработает пустые GT для отдельных картинок.
            continue

        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 5:
                    continue
                cls = int(parts[0])
                xc, yc, w, h = map(float, parts[1:5])

                rows.append({
                    'image_id': image_id,
                    'label': cls,
                    'xc': xc,
                    'yc': yc,
                    'w': w,
                    'h': h,
                    'w_img': w_img,
                    'h_img': h_img,
                    'score': 1.0,       # Для GT можно поставить 1.0 или np.nan
                    'time_spent': 0.0   # Для GT 0.0, не используется в метрике
                })

    if not rows:
        raise RuntimeError('GT пустой — в labels нет объектов. '
                           'Скрипт метрики потребует непустой gt_df.')

    gt_df = pd.DataFrame(rows, columns=COLUMNS)
    gt_df.to_csv(out_csv, index=False)
    print(f'GT CSV saved to: {out_csv}  |  rows: {len(gt_df)}')
    return gt_df

## ДАлее код уже расчета GT, prediction, metric

In [8]:
# Пути
DATASET_VAL = '/mnt/data/dataset/03_validation__DataSet_Human_Rescue/private'
# DATASET_VAL = '/mnt/data/dataset/03_validation__DataSet_Human_Rescue/public'

GT_CSV_PATH = './ground_truth.csv'
PRED_CSV_PATH = './prediction.csv'

In [9]:
# 0) Строим GT
# gt_df = build_gt_csv_from_yolo(DATASET_VAL, GT_CSV_PATH)

In [10]:
%%time
# 1) строим наши prediction.csv
predicted_df = process_images(DATASET_VAL, PRED_CSV_PATH)

2025-08-09 12:50:40,229 | INFO | [profile] windows=1 | tiles=1 | windows_ms=27.8 | tiles_ms=0.3 | gpu_infer=71.9 | decode=0.3 | nms=0.0 | total=100.4 ms
2025-08-09 12:50:40,230 | INFO | [profile-extra] batches=1 batch_size=8
2025-08-09 12:50:41,500 | INFO | [profile] windows=35 | tiles=35 | windows_ms=99.4 | tiles_ms=1.6 | gpu_infer=1032.3 | decode=2.5 | nms=0.0 | total=1135.9 ms
2025-08-09 12:50:41,501 | INFO | [profile-extra] batches=5 batch_size=8
2025-08-09 12:50:41,777 | INFO | [profile] windows=3 | tiles=3 | windows_ms=95.3 | tiles_ms=0.1 | gpu_infer=122.3 | decode=0.2 | nms=0.1 | total=218.0 ms
2025-08-09 12:50:41,777 | INFO | [profile-extra] batches=1 batch_size=8
2025-08-09 12:50:42,037 | INFO | [profile] windows=3 | tiles=3 | windows_ms=89.2 | tiles_ms=0.1 | gpu_infer=116.8 | decode=0.2 | nms=0.1 | total=206.4 ms
2025-08-09 12:50:42,037 | INFO | [profile-extra] batches=1 batch_size=8
2025-08-09 12:50:42,282 | INFO | [profile] windows=2 | tiles=2 | windows_ms=89.1 | tiles_ms=0

Обработка выборки выполнена успешно!
CPU times: user 17min 6s, sys: 3min 41s, total: 20min 47s
Wall time: 4min 20s


In [11]:
# 2) считаем на коде огров evaluate
from metric import open_df_as_bytes, evaluate #их фаил

pred_bytes = open_df_as_bytes(PRED_CSV_PATH)
gt_bytes   = open_df_as_bytes(GT_CSV_PATH)

# 3) m — число изображений в валидации
import os
img_dir = Path(DATASET_VAL) / 'images'
# считаем только по допустимым расширениям
m = len([p for p in img_dir.iterdir() if p.suffix.lower() in [e.lower() for e in IMAGE_EXTS]])
print(m)
metric, accuracy, fp_rate, avg_time = evaluate(
    predicted_file=pred_bytes,
    gt_file=gt_bytes,
    m=m,
    beta=1.0,
    thresholds=np.round(np.arange(0.3, 1.0, 0.07), 2),
    parallelize=True
)

print('Metric:', metric)
print('Accuracy:', accuracy)
print('FP rate:', fp_rate)
print('Avg time:', avg_time)

500
Metric: 0.6893164557
Accuracy: {0.3: 73.964, 0.37: 73.514, 0.44: 72.883, 0.51: 72.252, 0.58: 70.721, 0.65: 67.387, 0.72: 62.613, 0.79: 55.766, 0.86: 43.784, 0.93: 20.36}
FP rate: {0.3: 26.036, 0.37: 26.486, 0.44: 27.117, 0.51: 27.748, 0.58: 29.279, 0.65: 32.613, 0.72: 37.387, 0.79: 44.234, 0.86: 56.216, 0.93: 79.64}
Avg time: 0.415


In [ ]:
# 501
# Metric: 0.5919488818
# Accuracy: {0.3: 89.805, 0.37: 88.906, 0.44: 86.807, 0.51: 84.408, 0.58: 81.709, 0.65: 76.612, 0.72: 69.565, 0.79: 59.37, 0.86: 43.028, 0.93: 14.243}
# FP rate: {0.3: 10.195, 0.37: 11.094, 0.44: 13.193, 0.51: 15.592, 0.58: 18.291, 0.65: 23.388, 0.72: 30.435, 0.79: 40.63, 0.86: 56.972, 0.93: 85.757}
# Avg time: 0.027